In [ ]:
# this set of lines is to make sure that the python scripts can find the ROS libraries
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages')
sys.path.append('/usr/lib/python3/dist-packages')

import os

# change the ROS environment variable to th.lan.robolabe robot's IP address
ROBOT_HOSTNAME = 'robotics10' # for example, this could be an IP or a hostname depending on what is reachable on the network
CONSOLE_HOSTNAME = 'group10' # for example, notice that this is the IP of the computer running this script not the robot

# another example using hostnames is as follows
# ROBOT_HOSTNAME = 'robotics1' # for example, this could be an IP or a hostname depending on what is reachable on the network
# CONSOLE_HOSTNAME = 'group1' # for example, notice that this is the hostname of the computer running this script not the robot

# the following lines set up the networking variables for the scripts to run properly
os.environ['ROS_MASTER_URI'] = 'http://{}:11311'.format(ROBOT_HOSTNAME)
os.environ['ROS_HOSTNAME'] = CONSOLE_HOSTNAME
print('connected')

In [ ]:
# import python libraries
import rospy # ros library for python
from sensor_msgs.msg import Illuminance
from geometry_msgs.msg import Twist

# create a class to subscribe to sensor messages and show the data
class RoverIlluminanceController:
    SPIN_THRESHOLD = 300  # threshold to trigger spinning
    SPIN_SPEED = 1.0  # velocity for spinning
    STOP_SPEED = 0.0  # velocity for stopping
    
    
    def __init__(self):
        rospy.init_node('rover_illuminance_controller', anonymous=True)
        
        self.illuminance = 0.0
        self.twist = Twist()
        
        self.illuminance_sub = rospy.Subscriber('/rvr_driver/ambient_light', Illuminance, self.callback)
        self.cmd_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
        
        print('controller node started')       

    # callback function to receive the data
    def callback(self, msg):
        # print the data received
        self.illuminance = msg.illuminance
        print(f'Received illuminance: {self.illuminance}')
        
        #check if rover should be spinning
        self.update_rover_motion()
            
    def update_rover_motion(self):
        #updates motion based on the illuminance
        if self.illuminance >= self.SPIN_THRESHOLD:
            self.set_rover_spin(self.SPIN_SPEED)
        else:
            self.set_rover_spin(self.STOP_SPEED)
            
            
    def set_rover_spin(self, speed):
        #control rover spin
        self.twist.angular.z = speed
        self.cmd_vel_pub.publish(self.twist)
        print('rover spinning' if speed else 'rover stopped spinning')

    def run(self):
        rospy.spin()

if __name__ == '__main__':
    try:
        controller = RoverIlluminanceController()
        controller.run()
    except rospy.ROSInterruptException:
        ("Rover Illuminance Controller Node Interrupted")
    finally:
        rospy.loginfo("Node Shutdown")